In [1]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
alt.themes.enable('light')

ThemeRegistry.enable('light')

# Income Dist

Source: Earnings and hours worked, UK region by public and private sector: ASHE Table 25: time-series
https://www.ons.gov.uk/datasets/ashe-tables-25/editions/time-series/versions/7

## Full

In [60]:
# Earnings and hours worked, UK region by public and private sector: ASHE Table 25: time-series
df = pd.read_excel("ashe-tables-25-time-series-v7-filtered-2024-06-30T15-09-49Z.xlsx", "Dataset", skiprows=2)
df = df[['AveragesAndPercentiles', '2023']]
df.columns = ['pctile', 'value']
df['pctile'] = df['pctile'].str.replace('Median', '50')
df['pctile'] = pd.to_numeric(df['pctile'], errors='coerce')/100
df = df.dropna()
df = df.sort_values('pctile')
dist_df = df.copy()

# Plot - full distribution
base = alt.Chart(df).encode(
    x=alt.X('pctile:Q', scale=alt.Scale(domain=(0, 1)), axis=alt.Axis(format='%')),
    y=alt.Y('value:Q', axis=alt.Axis(title='Gross annual pay (full-time), 2023', labelExpr="'£' + datum.label"),)
)

bars = base.mark_line(point={'filled': True, 'fill': '#E6224B'},color="#36B7B4")

median_rule = base.transform_filter(alt.datum.pctile == 0.5).mark_rule(color='#E6224B')

median_text = base.transform_filter(alt.datum.pctile == 0.5).mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=7,
    color='#E6224B',
    text=alt.expr("'Median: £' + format(datum.value, ',.0f')"),
)


chart = bars + median_rule + median_text

chart = chart.properties(
    width=450,
    height=300,
)

chart.save("fig1.png", scale_factor=3)

chart

alt.LayerChart(...)

## Restricted

In [61]:
df = dist_df.copy()

df['label'] = df.apply(lambda x: f"£{x['value']:,}", axis=1)

# Plot - full distribution
base = alt.Chart(df).encode(
    x=alt.X('pctile:Q', scale=alt.Scale(domain=(0, 1)), axis=alt.Axis(format='%')),
    y=alt.Y('value:Q', axis=alt.Axis(title='Gross annual pay (full-time), 2023', labelExpr="'£' + datum.label"),)
)

bars = base.mark_line(point={'filled': True, 'fill': '#E6224B'},color="#36B7B4")

pctile_rule = base.transform_filter(filter="datum.pctile == 0.5 | datum.pctile == 0.4").mark_rule(color='#F4C245')

pctile_text = base.transform_filter(filter="datum.pctile == 0.5 | datum.pctile == 0.4").mark_text(
    align=alt.expr("if(datum.pctile == 0.5, 'left', 'right')"),
    baseline='middle',
    dx=7,
    dy=alt.expr("if(datum.pctile == 0.5, 7, -12)"),
    color='#F4C245',
    text=alt.expr("'£'+format(datum.value, ',.0f')"),
)


chart = bars + pctile_rule + pctile_text

chart = chart.properties(
    width=450,
    height=300,
)

chart.save("fig2.png", scale_factor=3)
chart


/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [57]:
df

,pctile,value,label
0,0.10,21000,"£21,000.0"
1,0.20,24496,"£24,496.0"
2,0.25,26085,"£26,085.0"
3,0.30,27673,"£27,673.0"
4,0.40,31069,"£31,069.0"
11,0.50,34963,"£34,963.0"
5,0.60,39516,"£39,516.0"
6,0.70,44738,"£44,738.0"
7,0.75,48000,"£48,000.0"
8,0.80,52007,"£52,007.0"


In [58]:
(31069+34963)/2

33016.0

## With £1 of MW

low pay estimates from the Annual Survey of Hours and Earnings (ASHE)
https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/distributionoflowpaidjobsby10pbands

In [112]:
df = pd.read_excel("10pbands2023.xls", sheet_name="2023 provisional", skiprows=4)
df = df.iloc[:97, :5]
df.columns = ['amount', '16-17', '18-20', '21-22', '23+']
df = df.melt(id_vars='amount', var_name='age', value_name='count')
df['count'] = pd.to_numeric(df['count'], errors='coerce')
df = df.dropna()
df = df.pivot_table(index='amount', columns='age', values='count').reset_index()
df['all'] = df['16-17'] + df['18-20'] + df['21-22'] + df['23+']
df = df[['amount', 'all']].dropna()

number_of_all_employees = df.iloc[-1]['all']
df = df.iloc[:-1]
df['all'] = df['all']/number_of_all_employees
df

base = alt.Chart(df).encode(
    x=alt.X('amount:Q', axis=alt.Axis(title='Hourly Rate (2023)', labelExpr="'£' + datum.label"),),
    y=alt.Y('all:Q', axis=alt.Axis(title='Proportion of all employees making less', format='%'), scale=alt.Scale())
)

bars = base.mark_area(interpolate='step-after', fill='#36B7B4', line={'color': '#36B7B4'},)

NLW_rule = alt.Chart(pd.DataFrame({'amount': [10.42], 'all': [0]})).mark_rule(
    strokeDash=[3, 5],
    color='#E6224B').encode(x='amount:Q')

NLW_text = alt.Chart(pd.DataFrame({'amount': [10.42], 'all': [0], 'label': ['NLW: £10.42']})).mark_text(
    align='left',
    dx=3,
    dy=7,
    baseline='middle',
    color='#E6224B',
).encode(
    x='amount:Q',
    y=alt.value(0.5),
    text='label',
)

upper_rule = alt.Chart(pd.DataFrame({'amount': [11.42], 'all': [0]})).mark_rule(
    strokeDash=[3, 5],
    color='#E6224B').encode(x='amount:Q')

upper_text = alt.Chart(pd.DataFrame({'amount': [11.42], 'all': [0], 'label': ['NLW + £1']})).mark_text(
    align='left',
    dx=3,
    dy=7,
    baseline='middle',
    color='#E6224B',
).encode(
    x='amount:Q',
    y=alt.value(0.5),
    text='label',
)

chart = bars + NLW_rule + NLW_text + upper_rule + upper_text

chart = chart.properties(
    width=500,
    height=300,
)

chart.save("fig3.png", scale_factor=3)
chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [111]:
df

age,amount,all
34,7.3,0.007878
35,7.4,0.008118
36,7.49,0.008666
37,7.5,0.010549
38,7.6,0.011817
39,7.7,0.012193
40,7.8,0.012604
41,7.9,0.013015
42,8.0,0.013564
43,8.1,0.014934
